## 2023638 | Anton Dementyev | Coursework 1: Algorithm Implementation

In [2]:
import csv
import datetime
import math
import threading
import pprint
from itertools import chain, combinations
from collections import defaultdict
from timeit import default_timer as timer

### Basic I/O

In [3]:
def read_file(filename, skipEmptyColumn):
    ds = []
    with open(filename, newline='') as csvfile:
        filereader = csv.reader(csvfile, delimiter=',')
        if skipEmptyColumn:
            for row in filereader:
                ## tuple is hashable, which is useful for all further actions
                ## sorting by name is default
                ds.append(tuple(sorted(row[:-1])))
        else:
            for row in filereader:
                ds.append(tuple(sorted(row)))
        return ds

<span style="color:gray">Testing Read File</span>

In [4]:
read = read_file('GroceryStore.csv', True)[:5]
print(read)

[('Butter', 'Cheese', 'Coffee Powder', 'Ghee', 'Lassi', 'Yougurt'), ('Coffee Powder', 'Ghee'), ('Butter', 'Cheese', 'Lassi', 'Tea Powder'), ('Bread', 'Butter', 'Cheese', 'Coffee Powder', 'Panner', 'Tea Powder'), ('Butter', 'Cheese', 'Coffee Powder', 'Sugar', 'Sweet', 'Yougurt')]


In [5]:
def write_file(data, filename, reverse):
    # reverse mode can be helpful to create a required order of itemsets
    if reverse:
        data.reverse()
    if data != [] and data is not None:
        with open(filename, 'w', newline='') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',')
            for row in data:
                filewriter.writerow(row)
    else:
        print('Attempting to write empty dataset')

#### ________________________________________________________________________

### Apriori Algorithm

In [6]:
def calculate_support_count(instance, data, is_set):
    count = 0
    # if item is a plain string, do not waste memory to make convert it to a set
    if is_set:
        for row in data:
            if set(instance).issubset(set(row)): 
                count = count + 1
    else:
        for row in data:
            if instance in row:
                count = count + 1
    return count

In [7]:
def calculate_support(data, items):
    dct = {}
    lgtn = len(data)
    ## if not itemset is supplied, decompose the data to the mininum
    if items == None:
        for i in data:
            for j in i:
                if j in dct:
                    continue
                else:
                    support_count = calculate_support_count(j, data, False)
                    dct[j] = support_count / lgtn
    else:
        for i in items:
            support_count = calculate_support_count(i, data, True)
            dct[i] = support_count / lgtn
    return dct

In [8]:
def support_elimination(data, items, minimal_support, support_dict):
    ## a support dictitionary can already be supplied, if not - calculate
    dct = calculate_support(data, items) if support_dict is None else support_dict
    support_resistant = []
    for key in dct:
        if dct[key] >= minimal_support:
            support_resistant.append(key)
    return support_resistant

In [9]:
def generate_candidates_fk1_1(previous_step_f_itemset, step_one_f_itemset):
    lst = []
    if isinstance(previous_step_f_itemset[0], list) or isinstance(previous_step_f_itemset[0], tuple): 
        for i in previous_step_f_itemset:
            for k in step_one_f_itemset:
                if k not in i:
                    lst.append(tuple(sorted([*i, k])))
    else:
        for i in previous_step_f_itemset:
            for k in step_one_f_itemset:
                if k != i:
                    lst.append(tuple(sorted((i, k))))
    return set(lst)

<span style="color:gray">Testing Candidates Generation</span>

In [12]:
data = read_file('GroceryStore.csv', True)
unique = calculate_support(data, None)
el_one_unique = support_elimination(data, None, 0.2, unique)
el_one_unique = tuple(sorted([i for i in el_one_unique]))
print(generate_candidates_fk1_1(el_one_unique, el_one_unique), end=" ")

{('Bread', 'Sugar'), ('Cheese', 'Milk'), ('Lassi', 'Sweet'), ('Ghee', 'Lassi'), ('Butter', 'Sweet'), ('Milk', 'Sweet'), ('Lassi', 'Panner'), ('Butter', 'Panner'), ('Milk', 'Panner'), ('Coffee Powder', 'Lassi'), ('Bread', 'Milk'), ('Coffee Powder', 'Ghee'), ('Lassi', 'Tea Powder'), ('Ghee', 'Sugar'), ('Bread', 'Butter'), ('Lassi', 'Yougurt'), ('Milk', 'Yougurt'), ('Butter', 'Tea Powder'), ('Milk', 'Tea Powder'), ('Cheese', 'Sweet'), ('Panner', 'Sugar'), ('Panner', 'Sweet'), ('Butter', 'Yougurt'), ('Coffee Powder', 'Sugar'), ('Cheese', 'Panner'), ('Ghee', 'Milk'), ('Bread', 'Sweet'), ('Cheese', 'Tea Powder'), ('Panner', 'Tea Powder'), ('Cheese', 'Yougurt'), ('Bread', 'Panner'), ('Coffee Powder', 'Milk'), ('Panner', 'Yougurt'), ('Bread', 'Tea Powder'), ('Butter', 'Cheese'), ('Bread', 'Yougurt'), ('Butter', 'Coffee Powder'), ('Ghee', 'Sweet'), ('Butter', 'Lassi'), ('Butter', 'Ghee'), ('Ghee', 'Panner'), ('Lassi', 'Sugar'), ('Coffee Powder', 'Sweet'), ('Ghee', 'Tea Powder'), ('Sugar', 'Swee

In [13]:
def apriori_algorithm(filename, max_length, is_fixed_length, min_support):
    data = read_file(filename, True)
    ## a trick in order not to calculate support for 1-itemsets two times
    one_unique_dict = calculate_support(data, None)
    el_one_unique = support_elimination(data, None, min_support, one_unique_dict)
    el_k_unique = tuple(sorted([i for i in el_one_unique]))
    ## if only n-length itemsets
    if is_fixed_length:
        for i in range(max_length-1):
            if el_k_unique == []:
                return []
            k_unique = generate_candidates_fk1_1(el_k_unique, el_one_unique)
            el_k_unique = support_elimination(data, k_unique, min_support, None)
        return el_k_unique
    ## if 2 - n-length itemsets
    else:
        frequent = []
        for i in range(max_length-1):
            if el_k_unique == []:
                return frequent
            k_unique = generate_candidates_fk1_1(el_k_unique, el_one_unique)
            el_k_unique = support_elimination(data, k_unique, min_support, None)
            frequent.extend(el_k_unique)
        return frequent

<span style="color:gray">Testing Apriori</span>

In [15]:
dataset_name = 'GroceryStore.csv'
min_support = 0.1
max_itemset_length = 3
is_fixed_length = False

start = timer()
apriori_frequent = apriori_algorithm(dataset_name, max_length=max_itemset_length, is_fixed_length=is_fixed_length, min_support=min_support)
end = timer()
print(f'Apriori algorithm took appx. {"{:.2f}".format(end - start)}s to run on the {dataset_name} dataset')

Apriori algorithm took appx. 2.16s to run on the GroceryStore.csv dataset


In [811]:
write_file(apriori_frequent, f'Apriori-min_support={min_support}-{datetime.datetime.now()}.csv', reverse=True)

<h5 style="color: #7d3434">NB: in the next sections some functions from the current section will be reused</h5>

#### ________________________________________________________________________

### Association Rule

In [16]:
def association_rule_props(itemset, data):
    T = len(data)
    support_count_dict = {}
    gen_dict = {}
    for item in itemset:
        if item not in support_count_dict:
            ## calculate support for each set of objects
            support_count_dict[item] = calculate_support_count(item, data, True)
        ## create all possible subsets of the item / set of objects
        all_subsets = chain.from_iterable(combinations(item, i) for i in range(1, len(item)))
        for x in all_subsets:
            if x not in support_count_dict:
                support_count_dict[x] = calculate_support_count(x, data, True)
            ## {Y} is {X, Y} - {X}
            y = tuple(set(item).difference(set(x)))
            if not y in support_count_dict:
                support_count_dict[y] = calculate_support_count(y, data, True)
            if not (x, y) in gen_dict:
                support_count = support_count_dict[item]
                support_count_x = support_count_dict[x]
                support = support_count / T
                support_x = support_count_x / T
                ## return props for each X => Y pair
                gen_dict[(x, y)] = {
                    'support': support,
                    'confidence': support_count / support_count_x,
                    'lift': support / (support_x * support_count_dict[y] / T)
                }
    return gen_dict

In [17]:
def association_rule(min_support, min_confidence, data_file_name, frequent_file_name):
    # read the main file and frequent itemsets
    data = read_file(data_file_name, skipEmptyColumn=True)
    frequent = read_file(frequent_file_name, skipEmptyColumn=False)
    characteristics = association_rule_props(frequent, data)
    ## eliminate based on the supplied params
    return {k: v for k, v in characteristics.items() if v['support'] >= min_support and v['confidence'] >= min_confidence}

<span style="color:gray">Testing Association Rule</span>

In [18]:
min_support = 0.1
min_confidence = 0.468
association_rule(min_support, min_confidence, dataset_name, 'Apriori-min_support=0.1-2021-03-21 00:45:30.864484.csv')

{(('Lassi', 'Panner'), ('Sweet',)): {'support': 0.10098994092288041,
  'confidence': 0.5066079295154186,
  'lift': 1.1573538072424097},
 (('Lassi', 'Sweet'), ('Panner',)): {'support': 0.10098994092288041,
  'confidence': 0.49107142857142855,
  'lift': 1.129897265666002},
 (('Panner', 'Sweet'), ('Lassi',)): {'support': 0.10098994092288041,
  'confidence': 0.5049900199600799,
  'lift': 1.1644891366016128},
 (('Lassi',), ('Sweet',)): {'support': 0.20565224333386556,
  'confidence': 0.4742268041237113,
  'lift': 1.0833786154392868},
 (('Sweet',), ('Lassi',)): {'support': 0.20565224333386556,
  'confidence': 0.4698157942732081,
  'lift': 1.0833786154392864},
 (('Butter',), ('Sugar',)): {'support': 0.20525307360689765,
  'confidence': 0.4690749863163656,
  'lift': 1.071804684166143},
 (('Sugar',), ('Butter',)): {'support': 0.20525307360689765,
  'confidence': 0.4689894199197373,
  'lift': 1.071804684166143},
 (('Panner',), ('Bread',)): {'support': 0.20357656075363245,
  'confidence': 0.46840

#### ________________________________________________________________________

### FP-Growth Algorithm

In [19]:
## tree node
class node:
    def __init__(self, entity, parent, count):
        self.entity = entity ## item name
        self.parent_node = parent
        self.child_nodes = {} ## children nodes with the key of item name
        self.count = count ## number of passes of the node

In [20]:
def fp_tree(data, min_support):
    one_unique_dict = calculate_support(data, None)
    el_one_unique = support_elimination(data, None, min_support, one_unique_dict)
    ## sort by support, descending order
    el_one_unique_sorted = [i for i in dict(sorted(one_unique_dict.items(), key=lambda item: item[1], reverse=True))]
    ## sorted linked lists of tail nodes
    node_links = {}
    for i in el_one_unique_sorted:
        ## the list itself
        node_links[i] = []
    null_node = node(None, None, 1)
    for item in data:
        ## sorting items based on the order of unique 1-itemsets
        item_ordered = sorted(item, key=lambda x: el_one_unique_sorted.index(x))
        ## root of the tree
        start_node = null_node
        for nd in item_ordered:
            ## if the path has been created, just update the count
            if nd in start_node.child_nodes:
                start_node.child_nodes[nd].count += 1
                start_node = start_node.child_nodes[nd]
            else:
                ## create a new node if there was not path
                this_node = node(nd, start_node, 1)
                start_node.child_nodes[nd] = this_node
                start_node = this_node
                ## update the "linked list"
                node_links[nd].append(this_node) 
    ## sort dict based on frequency
#     node_links = {k: v for k, v in sorted(node_links.items(), key=lambda x: el_one_unique_sorted.index(x[0]))}
    return (null_node, node_links)

In [21]:
def fp_frequent_itemsets(node_links, min_support_count):
    lst = []
    for key in node_links:
        ## this is the list of tail nodes
        patterns = defaultdict(int)
        freq = []
        ## for each node in the linked list
        for nd in node_links[key]:
            current = nd
            prefix = []
            ## go up until reaches null node
            while current.parent_node != None:
                prefix.append(current.entity)
                current = current.parent_node
            ## create all possible links in a branch, 1-itemsets are EXCLUDED
            all_subsets = chain.from_iterable(combinations(prefix, i) for i in range(2, len(prefix)+1))
            ## put in the buffer dict
            for i in all_subsets:
                patterns[i] += nd.count
                if patterns[i] >= min_support_count:
                    ## if it reaches the min support count, put it in the frequent list and make sure it doesn't get put there again
                    ## the latter is ensure by making the count -infinity
                    patterns[i] = -math.inf 
                    freq.append(i)
            lst.extend(freq)
    return set(lst)

In [22]:
def fp_growth(filename, max_length, is_fixed_length, min_support):
    data = read_file(filename, True)
    tree, node_links = fp_tree(data, min_support)
    min_support_count = min_support * len(data)
    ## param-based return, implemented to match apriori function
    if max_length is None:
        return fp_frequent_itemsets(node_links, min_support_count)
    else:
        if is_fixed_length:
            return [x for x in fp_frequent_itemsets(node_links, min_support_count) if len(x) == max_length]
        else:
            return [x for x in fp_frequent_itemsets(node_links, min_support_count) if len(x) <= max_length]

<span style="color:gray">Testing FP</span>

In [23]:
start = timer()
fp = fp_growth('GroceryStore.csv', max_length=5, is_fixed_length=True, min_support=0.0235)
end = timer()
print(f'FP-Growth algorithm took appx. {"{:.2f}".format(end - start)}s to run on the {dataset_name} dataset')

FP-Growth algorithm took appx. 0.41s to run on the GroceryStore.csv dataset


<span style="color:gray">Testing FP time: ~ the most demanding case for this dataset</span>

In [24]:
start = timer()
fp = fp_growth('GroceryStore.csv', max_length=None, is_fixed_length=False, min_support=0.005)
end = timer()
print(f'FP-Growth algorithm took appx. {"{:.2f}".format(end - start)}s to run on the {dataset_name} dataset')

FP-Growth algorithm took appx. 0.66s to run on the GroceryStore.csv dataset


<span style="color:gray">Writing into a file</span>

In [883]:
fp = sorted(fp, key=len, reverse=True)
write_file(fp, f'FP-Growth-min_support={min_support}-{datetime.datetime.now()}.csv', reverse=False)

#### ________________________________________________________________________

### Experiments on the Dataset
##### Conducted with both Apriori and FP-Growth. All the parameters can be inserted in the Algorithm functions and run again.

<div style="color:gray">
    <span>Judging by the lift, we can see that after purchasing Panner & Sweet customers are more likely to purchase Lassi.</span></br></br>
    <span>Params:</span>
    <ul>
        <li>min_support = 0.1</li>
        <li>max_itemset_length = 3</li>
        <li>is_fixed_length = False</li>
    </ul>
</div>

In [25]:
association_rule(min_support=0, min_confidence=0.49, 
                 data_file_name=dataset_name, frequent_file_name='Apriori-min_support=0.1-2021-03-21 00:45:30.864484.csv')

{(('Lassi', 'Panner'), ('Sweet',)): {'support': 0.10098994092288041,
  'confidence': 0.5066079295154186,
  'lift': 1.1573538072424097},
 (('Lassi', 'Sweet'), ('Panner',)): {'support': 0.10098994092288041,
  'confidence': 0.49107142857142855,
  'lift': 1.129897265666002},
 (('Panner', 'Sweet'), ('Lassi',)): {'support': 0.10098994092288041,
  'confidence': 0.5049900199600799,
  'lift': 1.1644891366016128}}

<div style="color:gray">
    <span>For 4-itemsets we can see that Panner, Sweet and Tea Powder are to be put together, especially if the task is to increase Lassi sales.</span></br></br>
    <span>Params:</span>
    <ul>
        <li>min_support = 0.048</li>
        <li>max_itemset_length = 4</li>
        <li>is_fixed_length = True</li>
    </ul>
</div>

In [26]:
association_rule(min_support=0, min_confidence=0.52, 
                 data_file_name=dataset_name, frequent_file_name='Apriori-min_support=0.048-2021-03-20 23:12:47.216416.csv')

{(('Milk', 'Panner', 'Sweet'), ('Lassi',)): {'support': 0.04941721219862685,
  'confidence': 0.5254668930390493,
  'lift': 1.21170808214417},
 (('Lassi', 'Panner', 'Tea Powder'),
  ('Sweet',)): {'support': 0.04981638192559476, 'confidence': 0.5324232081911263, 'lift': 1.2163292186398043},
 (('Panner', 'Sweet', 'Tea Powder'),
  ('Lassi',)): {'support': 0.04981638192559476, 'confidence': 0.5342465753424658, 'lift': 1.231953719208344}}

<div style="color:gray">
    <span>Tea Powder, Coffee Powder and Milk (ranked to highest P to lowest) are most likely to go in the basket with Panner when a customer buys Lassi and Sweet.</span></br></br>
    <span>Params:</span>
    <ul>
        <li>min_support = 0.0235</li>
        <li>max_itemset_length = 5</li>
        <li>is_fixed_length = True</li>
    </ul>
</div>

In [27]:
min_confidence = 0.53
rule = association_rule(min_support=0, min_confidence=min_confidence, 
                 data_file_name=dataset_name, frequent_file_name='FP-Growth-min_support=0.02-2021-03-22 16:52:48.530205.csv')
pprint.pprint(rule)

{(('Coffee Powder', 'Lassi', 'Panner', 'Tea Powder'), ('Sweet',)): {'confidence': 0.5376344086021505,
                                                                    'lift': 1.2282342881908694,
                                                                    'support': 0.023950183618074404},
 (('Coffee Powder', 'Panner', 'Sweet', 'Tea Powder'), ('Lassi',)): {'confidence': 0.5494505494505495,
                                                                    'lift': 1.2670135461004386,
                                                                    'support': 0.023950183618074404},
 (('Lassi', 'Milk', 'Panner', 'Tea Powder'), ('Sweet',)): {'confidence': 0.5314183123877917,
                                                           'lift': 1.2140335183238153,
                                                           'support': 0.02363084783650008},
 (('Milk', 'Panner', 'Sweet', 'Tea Powder'), ('Lassi',)): {'confidence': 0.5323741007194245,
                                   

In [925]:
## we can write the acquired rules in a file
write_file(rule.keys(), f'Association_rule-confidence={min_confidence}-{datetime.datetime.now()}.csv', reverse=False)